In [12]:
print("Hello")

Hello


In [22]:
"""
Below is an example script to reproduce, in a simplified manner,
the Motor Imagery benchmark results of the Augmented Covariance
Method (ACM). It uses the MOABB library with the BNCI2014-001 dataset.

Step-by-step notes (in comments) show how to handle:
1) Loading the dataset and optionally filtering subjects.
2) Configuring a MotorImagery paradigm with band-pass filtering.
3) Building pipelines for classic approaches and ACM-based approaches.
4) Setting up parameter grids for possible nested cross-validation.
5) Running a WithinSessionEvaluation or CrossSessionEvaluation without
   passing unsupported arguments (e.g., 'subj_list' is not recognized by
   the latest MOABB code).
6) Collecting and displaying the results.

Make sure to install:
    pip install moabb mne numpy scipy scikit-learn pyriemann
as well as the code for the Augmented Covariance if needed.

This script addresses a potential TypeError when creating a
WithinSessionEvaluation or CrossSessionEvaluation, by not passing
'args' that are no longer recognized ('subj_list'). Instead, if you
want to restrict which subjects are processed, you can directly
edit dataset.subject_list before running the evaluation.
"""

import numpy as np
import pandas as pd

# MOABB imports
from moabb.datasets import BNCI2014_001
from moabb.paradigms import MotorImagery
from moabb.evaluations import WithinSessionEvaluation, CrossSessionEvaluation

# sklearn and pyriemann imports
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from pyriemann.estimation import Covariances
from pyriemann.classification import MDM, FgMDM
from pyriemann.tangentspace import TangentSpace
from pyriemann.spatialfilters import CSP

# If you are using the Augmented Covariance approach from a custom module,
# import the class from wherever you've placed it. For illustration:
try:
    from moabb.pipelines.features import AugmentedDataset
except ImportError:
    # As a placeholder, define a dummy AugmentedDataset if not available.
    # Replace with the actual AugmentedDataset class from the relevant source.
    class AugmentedDataset:
        def __init__(self, order=2, lag=1):
            self.order = order
            self.lag = lag
        def fit_transform(self, X, y=None):
            return X
        def transform(self, X):
            return X


###############################################################################
# 1) Configure general parameters and bandpass
###############################################################################

# Filter parameters
FMIN = 8.0
FMAX = 35.0

# Focus on 2-class MI, for instance left vs right hand
EVENTS = ["left_hand", "right_hand"]  # 2-class

# Time range of the epoch
TMIN = 2.0
TMAX = 6.0

###############################################################################
# 2) Load the dataset and (optionally) choose a subset of subjects
###############################################################################
dataset = BNCI2014_001()

# Example: if you only want certain subjects, you can do:
dataset.subject_list = [1]  # etc.
# By default, uses dataset.subject_list as provided.

# Create the MotorImagery paradigm
paradigm = MotorImagery(
    events=EVENTS,
    n_classes=len(EVENTS),
    fmin=FMIN,
    fmax=FMAX,
    tmin=TMIN,
    tmax=TMAX
)

###############################################################################
# 3) Build pipelines
###############################################################################

# Baseline pipeline 1: Covariance + Tangent Space + SVM
cov_tang_svm = Pipeline([
    ("Cov", Covariances(estimator="scm")),
    ("Tangent", TangentSpace(metric="riemann")),
    ("SVM", SVC(kernel="rbf"))
])

# # Baseline pipeline 2: CSP + LDA
# csp_lda = Pipeline([
#     ("CSP", CSP(nfilter=4, log=True)),
#     ("LDA", LDA(solver="lsqr", shrinkage="auto"))
# ])

# # Baseline pipeline 3: FgMDM
# fgmdm = Pipeline([
#     ("Cov", Covariances(estimator="scm")),
#     ("FgMDM", FgMDM(metric="riemann"))
# ])

# # Baseline pipeline 4: Cov + logistic regression
# cov_lr = Pipeline([
#     ("Cov", Covariances(estimator="scm")),
#     ("Tangent", TangentSpace(metric="riemann")),
#     ("LR", LogisticRegression(max_iter=1000))
# ])

# ACM pipeline: Augmented Covariance + Tangent Space + SVM
acm_tang_svm = Pipeline([
    ("ACM", AugmentedDataset()),  # Use actual augmented transformation
    ("Cov", Covariances(estimator="lwf")),
    ("Tangent", TangentSpace(metric="riemann")),
    ("SVM", SVC(kernel="rbf"))
])

# ACM pipeline: Augmented Covariance + MDM
acm_mdm = Pipeline([
    ("ACM", AugmentedDataset()),
    ("Cov", Covariances(estimator="lwf")),
    ("MDM", MDM(metric=dict(mean='riemann', distance='riemann')))
])

pipelines = {
    # "CovTangSVM": cov_tang_svm,
    # "CSP_LDA": csp_lda,
    # "FgMDM": fgmdm,
    # "CovLR": cov_lr,
    "ACM_TangSVM": acm_tang_svm,
    "ACM_MDM": acm_mdm
}

###############################################################################
# 4) Parameter grid for (nested) cross-validation for hyperparam tuning
###############################################################################
param_grid = {
    # "CovTangSVM": {
    #     "Tangent__metric": ["riemann"],
    #     "SVM__C": [0.5, 1, 1.5],
    #     "SVM__kernel": ["linear", "rbf"]
    # },
    # "CSP_LDA": {
    #     "CSP__nfilter": [2, 4, 6, 8],
    # },
    # "FgMDM": {},
    # "CovLR": {
    #     "LR__C": [0.01, 0.1, 1, 10]
    # },
    "ACM_TangSVM": {
        "ACM__order": [1, 2, 3],
        "ACM__lag": [1, 2],
        "SVM__C": [0.5, 1, 1.5],
        "SVM__kernel": ["linear", "rbf"]
    },
    "ACM_MDM": {
        "ACM__order": [1, 2, 3],
        "ACM__lag": [1, 2]
    }
}

###############################################################################
# 5) Run Within-Session or Cross-Session evaluation
###############################################################################
# Note: do not pass 'subj_list' here; it's not a recognized argument.
# If you want to limit the subjects, filter dataset.subject_list beforehand.

# Example: Within-Session
# This performs k-fold CV (k=5 by default) within each session for each subject.
evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=[dataset],
    overwrite=False,
    random_state=42,
)

# Alternatively, Cross-Session:
# evaluation = CrossSessionEvaluation(
#     paradigm=paradigm,
#     datasets=[dataset],
#     overwrite=False,
#     random_state=42,
#     n_jobs=1
# )

###############################################################################
# 6) Execute the evaluation with nested cross-validation
###############################################################################
print("Starting nested evaluation. This may take some time...")
results = evaluation.process(
    pipelines=pipelines,
    # param_grid=param_grid, # triggers a nested CV using GridSearchCV
)

###############################################################################
# 7) Print or save the results
###############################################################################
print("\n--- RESULTS DataFrame ---")
print(results.head(20))

mean_scores = results.groupby(['pipeline'])['score'].mean()
std_scores = results.groupby(['pipeline'])['score'].std()

print("\n--- MEAN AND STD OF SCORES ---\n")
for pipe in mean_scores.index:
    print(f"{pipe}: {mean_scores[pipe]:.3f} +/- {std_scores[pipe]:.3f}")

# Export if desired:
# results.to_csv("motor_imagery_acm_results.csv", index=False)

print("\nDone. The results DataFrame contains details for each subject/session/fold.")


Starting nested evaluation. This may take some time...


BNCI2014-001-WithinSession:   0%|          | 0/1 [00:00<?, ?it/s]/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 23 events (all good), 4 – 8 s (baseline off), ~3.9 MB, data loaded,
 'left_hand': 11
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 23 events (all good), 4 – 8 s (baseline off), ~3.9 MB, data loaded,
 'left_hand': 11
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 23 events (all good), 4 – 8 s (baseline off), ~3.9 MB, data loaded,
 'left_hand': 11
 'right_hand': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: war

No hdf5_path provided, models will not be saved.
No hdf5_path provided, models will not be saved.


BNCI2014-001-WithinSession: 100%|██████████| 1/1 [00:03<00:00,  3.85s/it]



--- RESULTS DataFrame ---
       score      time  samples subject session  channels  n_sessions  \
0   0.750497  0.140671    140.0       1  0train        22           2   
1   0.686447  0.144405    141.0       1   1test        22           2   
2   0.617462  0.142078    140.0       1  0train        22           2   
3   0.652606  0.144038    141.0       1   1test        22           2   
4   0.645050  0.140228    141.0       2  0train        22           2   
5   0.617865  0.142475    141.0       2   1test        22           2   
6   0.704249  0.147674    141.0       3  0train        22           2   
7   0.746792  0.152483    141.0       3   1test        22           2   
8   0.600487  0.145376    141.0       4  0train        22           2   
9   0.495882  0.137823    140.0       4   1test        22           2   
10  0.751094  0.159001    141.0       5  0train        22           2   
11  0.643359  0.148901    141.0       5   1test        22           2   
12  0.573741  0.141534  